In [1]:
#Import required modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

In [2]:
#Create dataframe of all details of grants
grants = pd.merge(
    left = pd.read_csv('PublicAssistanceFundedProjectsDetails.csv').drop(columns = ['hash','lastRefresh','id']), 
    right = pd.read_csv('GeoPublicAssistanceApplicants.csv').drop(columns = ['hash','lastRefresh','id']), 
    how = 'inner', 
    left_on = ['applicantId', 'disasterNumber'], 
    right_on = ['applicantId', 'disasterNumber'], 
    suffixes = ('_grant', '_applicant')
)

In [3]:
#Remove negative grant amounts
grants.drop(index = grants[grants['totalObligated'] < 0].index, inplace = True)

In [4]:
#Trim the zip code suffix
grants['zipCode'] = grants['zipCode'].str.split("-", expand = True)[0]

In [5]:
#Add USGS Flood Event Name to grants dataframe
grants = pd.merge(
    left = grants, 
    right = pd.read_csv('Floods Listing - Flood IDs.csv')[['FEMA Disaster Code', 'FEMA Event Name', 'USGS Flood Event Name']], 
    left_on ='disasterNumber', 
    right_on = 'FEMA Disaster Code'
)

In [6]:
#Sort grants so highest grants show on plots
grants.sort_values(by = 'totalObligated', inplace = True)

In [7]:
#Plot distribution of funding for each event
for event in grants['USGS Flood Event Name'].unique():
    
    lats = grants[(grants['USGS Flood Event Name'] == event)]['latitude']
    longs = grants[(grants['USGS Flood Event Name'] == event)]['longitude']
    spacing = 2 * max(abs(lats.describe()['75%'] - lats.describe()['25%']), abs(longs.describe()['75%'] - longs.describe()['25%']))
    plt.figure(figsize = (12, 12))
    m = Basemap(projection = 'merc',
                llcrnrlat = lats.describe()['25%'] - spacing,
                urcrnrlat = lats.describe()['75%'] + spacing,
                llcrnrlon = longs.describe()['25%'] - spacing,
                urcrnrlon = longs.describe()['75%'] + spacing,
                lat_ts = 20,
                resolution ='l')
    m.drawcoastlines(color = 'gray')
    x, y = m(longs.values, lats.values)
    m.hexbin(x, 
             y, 
             C = grants[(grants['USGS Flood Event Name'] == event)]['totalObligated'], 
             gridsize = 500,
             cmap = 'inferno')
    
    plt.colorbar()
    plt.title(f'Distribution of FEMA Funding for {event}')
    plt.savefig(f'images/Hexbin Distribution of FEMA Funding for {event}.jpg')
    plt.close()

In [8]:
#Load depths data, remove outliers
depths = pd.read_csv('Hurricane.csv')
depths.drop(depths[depths['height_above_gnd'] > 20].index, inplace = True)

In [9]:
#Plot distribution of flooding for each event
for event in grants['USGS Flood Event Name'].unique():
    
    lats = grants[(grants['USGS Flood Event Name'] == event)]['latitude']
    longs = grants[(grants['USGS Flood Event Name'] == event)]['longitude']
    spacing = 2 * max(abs(lats.describe()['75%'] - lats.describe()['25%']), abs(longs.describe()['75%'] - longs.describe()['25%']))
    plt.figure(figsize = (12, 12))
    m = Basemap(projection = 'merc',
                llcrnrlat = lats.describe()['25%'] - spacing,
                urcrnrlat = lats.describe()['75%'] + spacing,
                llcrnrlon = longs.describe()['25%'] - spacing,
                urcrnrlon = longs.describe()['75%'] + spacing,
                lat_ts = 20,
                resolution ='l')
    m.drawcoastlines(color = 'gray')
    x, y = m(depths[depths['eventName'] == event]['longitude'].values, 
             depths[depths['eventName'] == event]['latitude'].values)
    m.hexbin(x, 
             y, 
             C = depths[depths['eventName'] == event]['height_above_gnd'], 
             gridsize = 100,
             cmap = 'YlGnBu')
    
    plt.colorbar()
    plt.title(f'Distribution of Flooding for {event}')
    plt.savefig(f'images/Hexbin Distribution of Flooding for {event}.jpg')
    plt.close()

In [10]:
#Plot cumulative ratio graph for Nate & Irma
plt.figure(figsize = (7,7))
for event in ['Nate October 2017', 'Irma September 2017']:
    a = np.cumsum(grants[(grants['USGS Flood Event Name'] == event)]['totalObligated'].sort_values(ascending = False)).values / np.sum(grants[(grants['USGS Flood Event Name'] == event)]['totalObligated'])
    b = np.arange(len(a)) / len(a)
    plt.plot(b, a, label = event)
plt.legend()
plt.title('Cumulative Ratio of Funding and Recepients, Hurricanes Nate and Irma');
plt.savefig('Cumulative Ratio of Funding and Recepients, Hurricanes Nate and Irma.jpg')
plt.close()

In [11]:
#Load ZIP-aggregated statistics
zip_stats =pd.read_csv('ZIPAggStats.csv').drop(columns = ['Unnamed: 0'])

In [12]:
#Scatterplot of funding and flood height by ZIP for Nate & Irma
plt.figure(figsize = (7, 5))
for event in ['Nate October 2017', 'Irma September 2017']:
    
    plt.scatter(zip_stats[zip_stats['USGS Flood Event Name'] == event]['height_above_gnd'], zip_stats[zip_stats['USGS Flood Event Name'] == event]['totalObligated'], label = event, alpha = 0.4)
plt.legend()
plt.title('Total Funding by ZIP vs Flood Depth by ZIP, Hurricanes Nate and Irma');
plt.savefig('Total Funding by ZIP vs Flood Depth by ZIP, Hurricanes Nate and Irma.jpg')
plt.close();

In [13]:
#Scatterplot of funding and flood height by ZIP for Nate & Irma
plt.figure(figsize = (7, 5))
for i, event in enumerate(['Nate October 2017', 'Irma September 2017']):
    
    plt.boxplot(zip_stats[zip_stats['USGS Flood Event Name'] == event]['funding_per_foot'].dropna(),
                positions = [i],
                labels = [event],
                widths = [0.4],
                vert = False)
plt.title('Funding per Foot Flooding, Hurricanes Nate and Irma')
plt.savefig('Funding per Foot Flooding, Hurricanes Nate and Irma.jpg')
plt.close();